In [1]:
import sqlite3
import pandas as pd

In [67]:
path = r"C:\Users\mches\MYPROJECT\amazon_reviews.db"
conn = sqlite3.connect(path)
cur = conn.cursor()

In [114]:
#This will be replaced with Dustin's SQL query, or we will start with a trimmed down SQLite file?
df = pd.read_sql_query("SELECT product_title, review_body, star_rating from books WHERE length(star_rating)<2 LIMIT 10000;", conn)
df

,product_title,review_body,star_rating
0,There Was an Old Lady Who Swallowed a Shell!,I love it and so does my students!,5
1,I Saw a Friend,My wife and I ordered 2 books and gave them as...,5
2,"Black Lagoon, Vol. 6",Great book just like all the others in the ser...,5
3,If I Stay,So beautiful,5
4,Stars 'N Strips Forever,Enjoyed the author's story and his quilts are ...,5
5,The Liar,Two or three pages into the book I suspected h...,2
6,Devil in the Details (Book 2: The Monastery Mu...,"&#34;Secrets in the Shallows,&#34; Book 1 of &...",5
7,Knowing When to Stop: A Memoir,I love it!,5
8,The American Pageant,It was a great purchase.,5
9,Punjabi C.L. Bible / Common Language Version,Quality product fast shipping.,5


In [115]:
from textblob import TextBlob
from numpy import interp

In [116]:
def sentiment(row):
    try:
        #raw text of review
        blob = TextBlob(row['review_body'])
        #sentiment polarity score
        polarity = blob.sentiment.polarity
        score = interp(polarity, [-1,1], [1,5])
        return score
    except:
        pass

In [117]:
def clean_sentiment(row):
    try:
        import string
        from nltk.corpus import stopwords

        #split raw text of review into tokens
        words= row['review_body']
        tokens = words.split()

        # remove punctuation from each token
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]

        #remove tokens that are not alphabetic
        tokens = [word for word in tokens if word.isalpha()]

        # filter out stop words
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]

        #convert this list to a string
        blob = ' '.join(tokens)

        #find sentiment of new string and convert to 1-5 scale
        blob = TextBlob(blob)
        polarity = blob.sentiment.polarity
        score = interp(polarity, [-1,1], [1,5])
        return score
    except:
        pass

In [118]:
df['sentiment_star_rating'] = df.apply(sentiment,axis=1)
df['cleaned_sentiment_star_rating'] = df.apply(clean_sentiment,axis=1)
df['star_rating']=df['star_rating'].astype(float)
df['difference'] = abs(df['star_rating'] - df['cleaned_sentiment_star_rating'])
df

,product_title,review_body,star_rating,sentiment_star_rating,cleaned_sentiment_star_rating,difference
0,There Was an Old Lady Who Swallowed a Shell!,I love it and so does my students!,5.0,4.250000,4.000000,1.000000
1,I Saw a Friend,My wife and I ordered 2 books and gave them as...,5.0,3.425016,3.636250,1.363750
2,"Black Lagoon, Vol. 6",Great book just like all the others in the ser...,5.0,4.600000,4.600000,0.400000
3,If I Stay,So beautiful,5.0,4.700000,4.700000,0.300000
4,Stars 'N Strips Forever,Enjoyed the author's story and his quilts are ...,5.0,4.400000,4.400000,0.600000
5,The Liar,Two or three pages into the book I suspected h...,2.0,3.336086,3.347024,1.347024
6,Devil in the Details (Book 2: The Monastery Mu...,"&#34;Secrets in the Shallows,&#34; Book 1 of &...",5.0,3.483333,3.425000,1.575000
7,Knowing When to Stop: A Memoir,I love it!,5.0,4.250000,4.000000,1.000000
8,The American Pageant,It was a great purchase.,5.0,4.600000,4.600000,0.400000
9,Punjabi C.L. Bible / Common Language Version,Quality product fast shipping.,5.0,3.400000,3.400000,1.600000


In [119]:
df['sentiment_star_rating'].mean()

3.710823573414274

In [120]:
df[df['review_body'].isnull()]

,product_title,review_body,star_rating,sentiment_star_rating,cleaned_sentiment_star_rating,difference
992,The Boleyn King: A Novel (The Boleyn Trilogy),None,5.0,NaN,NaN,NaN


In [121]:
#impute blank values with average sentiment scores 
df['sentiment_star_rating'] = df['sentiment_star_rating'].fillna(df['sentiment_star_rating'].mean())
df['cleaned_sentiment_star_rating'] = df['cleaned_sentiment_star_rating'].fillna(df['cleaned_sentiment_star_rating'].mean())

In [124]:
cur.close()
conn.close()